In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from tqdm import tqdm

import matplotlib.pyplot as plt

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
BATCH_SIZE = 128
EPOCHS = 15
LR = 1e-3
NUM_CLASSES = 10

In [5]:
stl_train = torchvision.datasets.STL10(root='./train',
                                       split='train',
                                       transform=transforms.ToTensor(),
                                       download=True)

stl_test = torchvision.datasets.STL10(root='./test',
                                      split='test',
                                      transform=transforms.ToTensor(),
                                      download=True)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./train/stl10_binary.tar.gz to ./train


  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./test/stl10_binary.tar.gz to ./test


In [6]:
train_meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x, _ in stl_train]
train_stdRGB = [np.std(x.numpy(), axis=(1,2)) for x, _ in stl_train]

train_meanR = np.mean([m[0] for m in train_meanRGB])
train_meanG = np.mean([m[1] for m in train_meanRGB])
train_meanB = np.mean([m[2] for m in train_meanRGB])
train_stdR = np.mean([s[0] for s in train_stdRGB])
train_stdG = np.mean([s[1] for s in train_stdRGB])
train_stdB = np.mean([s[2] for s in train_stdRGB])

In [7]:
train_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224, 224)),
                                    transforms.Normalize((train_meanR, train_meanG, train_meanB), (train_stdR, train_stdG, train_stdB)),
                                    transforms.RandomHorizontalFlip()
                                    ])

val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224, 224)),
                                    transforms.Normalize((train_meanR, train_meanG, train_meanB), (train_stdR, train_stdG, train_stdB)),
                                  ])

In [8]:
stl_train.transform = train_transform
stl_test.transform = val_transform

In [9]:
trainLoader = torch.utils.data.DataLoader(stl_train,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

testLoader = torch.utils.data.DataLoader(stl_test,
                                         batch_size=BATCH_SIZE,
                                         shuffle=False)

In [10]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super(conv_block, self).__init__()

        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs),
                                  nn.BatchNorm2d(out_channels),
                                  nn.ReLU())
    
    def forward(self, x):
        out = self.conv(x)
        return out

In [11]:
class inception_module(nn.Module):
    def __init__(self, in_ch, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        super(inception_module, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(in_ch, out_1x1, kernel_size=1),
                                   nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(in_ch, red_3x3, kernel_size=1),
                                   nn.Conv2d(red_3x3, out_3x3, kernel_size=3, padding=1),
                                   nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(in_ch, red_5x5, kernel_size=1),
                                   nn.Conv2d(red_5x5, out_5x5, kernel_size=5, padding=2),
                                   nn.ReLU())
        self.conv4 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                                   nn.Conv2d(in_ch, out_1x1pool, kernel_size=1),
                                   nn.ReLU())

    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2(x)
        out3 = self.conv3(x)
        out4 = self.conv4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

In [12]:
class aux_classifier(nn.Module):
    def __init__(self, in_channels):
        super(aux_classifier, self).__init__()

        self.conv = nn.Sequential(nn.AvgPool2d(kernel_size=5, stride=3),
                                 conv_block(in_channels, 128, kernel_size=1)
                                )
        
        self.fc = nn.Sequential(nn.Linear(2048, 1024),        
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(1024, NUM_CLASSES)
                               )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.flatten(start_dim=1)
        out = self.fc(x)
        return out

In [13]:
class My_GoogeLeNet(nn.Module):
    def __init__(self):
        super(My_GoogeLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception3a = inception_module(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = inception_module(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = inception_module(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = inception_module(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = inception_module(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = inception_module(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = inception_module(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception5a = inception_module(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = inception_module(832, 384, 192, 384, 48, 128, 128)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, NUM_CLASSES)

        self.aux1 = aux_classifier(512)
        self.aux2 = aux_classifier(528)

    def forward(self, x, train=True):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        if train:
            aux_out1 = self.aux1(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if train:
            aux_out2 = self.aux2(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)

        x = x.flatten(start_dim=1)
        x = self.dropout(x)
        out = self.fc(x)

        if train:
            return aux_out1, aux_out2, out
        else: 
            return out

In [14]:
model = My_GoogeLeNet().to(device)

In [15]:
summary(model, (3, 224, 224), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
         MaxPool2d-2           [-1, 64, 56, 56]               0
            Conv2d-3          [-1, 192, 56, 56]         110,784
         MaxPool2d-4          [-1, 192, 28, 28]               0
            Conv2d-5           [-1, 64, 28, 28]          12,352
              ReLU-6           [-1, 64, 28, 28]               0
            Conv2d-7           [-1, 96, 28, 28]          18,528
            Conv2d-8          [-1, 128, 28, 28]         110,720
              ReLU-9          [-1, 128, 28, 28]               0
           Conv2d-10           [-1, 16, 28, 28]           3,088
           Conv2d-11           [-1, 32, 28, 28]          12,832
             ReLU-12           [-1, 32, 28, 28]               0
        MaxPool2d-13          [-1, 192, 28, 28]               0
           Conv2d-14           [-1, 32,

In [16]:
# optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, 0.96)

In [17]:
def train_loop(model, trainLoader):
    model.train()
    train_loss = 0
    train_acc = 0
    for data, target in tqdm(trainLoader):
        data, target = data.to(device), target.to(device)
        aux_out1, aux_out2, output = model(data, train=True)
        aux_loss1 = loss_fn(aux_out1, target)
        aux_loss2 = loss_fn(aux_out2, target)
        loss = loss_fn(output, target)

        total_loss = aux_loss1 * 0.3 + aux_loss2 * 0.3 + loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        train_acc += (torch.argmax(output, dim=1) == target).sum().item()
    return train_loss / len(trainLoader), train_acc / len(trainLoader.dataset) * 100

In [18]:
def val_loop(model, valLoader):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for data, target in tqdm(valLoader):
            data, target = data.to(device), target.to(device)
            output = model(data, train=False)
            loss = loss_fn(output, target)

            val_loss += loss.item()
            val_acc += (torch.argmax(output, dim=1) == target).sum().item()
    return val_loss / len(valLoader), val_acc / len(valLoader.dataset) * 100

In [19]:
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = train_loop(model, trainLoader)
    val_loss, val_acc = val_loop(model, testLoader)

    print(f'\n[[ EPOCH {epoch:2d} ]]')
    print('Train Loss : {:.4f}, Train Accuracy : {:.2f} %'.format(train_loss, train_acc))
    print('Valid Loss : {:.4f}, Valid Accuracy : {:.2f} %\n'.format(val_loss, val_acc))

100%|██████████| 63/63 [00:40<00:00,  1.56it/s]



[[ EPOCH  1 ]]
Train Loss : 2.3195, Train Accuracy : 10.74 %
Valid Loss : 2.3074, Valid Accuracy : 10.00 %



100%|██████████| 63/63 [00:40<00:00,  1.55it/s]



[[ EPOCH  2 ]]
Train Loss : 2.2017, Train Accuracy : 14.58 %
Valid Loss : 2.0116, Valid Accuracy : 18.85 %



100%|██████████| 63/63 [00:40<00:00,  1.57it/s]


...


[[ EPOCH 13 ]]
Train Loss : 1.2162, Train Accuracy : 51.48 %
Valid Loss : 1.2731, Valid Accuracy : 49.15 %



100%|██████████| 63/63 [00:40<00:00,  1.54it/s]



[[ EPOCH 14 ]]
Train Loss : 1.1849, Train Accuracy : 52.80 %
Valid Loss : 1.2510, Valid Accuracy : 51.10 %



100%|██████████| 63/63 [00:40<00:00,  1.55it/s]


[[ EPOCH 15 ]]
Train Loss : 1.1334, Train Accuracy : 55.20 %
Valid Loss : 1.2498, Valid Accuracy : 52.48 %

